# Pawpularity Contest

Submissions are scored on the root mean squared error **RMSE**.

Guides to use:
*   Multi Input ==> https://www.kaggle.com/yaniv256/tensorflow-multi-input-pet-pawpularity-model
*   Transfer Learning ==> https://tfhub.dev/



In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Input
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import MeanSquaredError, MeanSquaredLogarithmicError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import backend as K
import math

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime, os

In [ ]:
tensorflow.test.gpu_device_name()

In [ ]:
input = Input(shape=(300, 300, 3))
x = Conv2D(filters=32, kernel_size=(5,5), activation='relu')(input)
x = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
x = MaxPool2D((2,2))(x)
x = Conv2D(filters=32, kernel_size=(5,5), activation='relu')(x)
x = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
x = MaxPool2D((2,2))(x)
x = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
x = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
x = MaxPool2D((2,2))(x)
x = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
x = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
x = MaxPool2D((2,2))(x)
x = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
x = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
x = MaxPool2D((2,2))(x)
x = Flatten()(x)
x = Dense(10, activation="relu")(x)
x = Dropout(0.2)(x)
output = Dense(1, activation="linear")(x)

model = Model(input, output)
model.summary()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive

In [ ]:
# !unzip petfinder-pawpularity-score.zip

In [ ]:
df = pd.read_csv('./train.csv')

In [ ]:
df.head()

In [ ]:
df['Id'] = df['Id'] + '.jpg';
df['Id']

In [ ]:
# For SGD we have to normalize data
# df['Pawpularity'] = df['Pawpularity'] / df['Pawpularity'].max()
df['Pawpularity']

In [ ]:
# You can see more informations here https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

# tf.keras.preprocessing.image.ImageDataGenerator(
#     featurewise_center=False, samplewise_center=False,
#     featurewise_std_normalization=False, samplewise_std_normalization=False,
#     zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0,
#     height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0,
#     channel_shift_range=0.0, fill_mode='nearest', cval=0.0,
#     horizontal_flip=False, vertical_flip=False, rescale=None,
#     preprocessing_function=None, data_format=None, validation_split=0.0, dtype=None
# )

train_dataGen = ImageDataGenerator(
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True,
    rotation_range = 10,
    shear_range = 10/180 * math.pi
)

# flow_from_dataframe(
#     dataframe, directory=None, x_col='filename', y_col='class',
#     weight_col=None, target_size=(256, 256), color_mode='rgb',
#     classes=None, class_mode='categorical', batch_size=32, shuffle=True,
#     seed=None, save_to_dir=None, save_prefix='',
#     save_format='png', subset=None, interpolation='nearest',
#     validate_filenames=True, **kwargs
# )

# target_size   tuple of integers (height, width), default: (256, 256). The dimensions to which all images found will be resized.
# class_mode    one of "binary", "categorical", "input", "multi_output", "raw", sparse" or None. "sparse": 1D numpy array of integer labels

# TODO: We will need to pass the data as array of strings in order to use sparse!!!!

train_gen = train_dataGen.flow_from_dataframe(
    dataframe=df,
    directory='./train',
    x_col='Id',
    y_col='Pawpularity',
    target_size=(300, 300), 
    color_mode='rgb',
    class_mode='raw',
    batch_size=32
)

def scheduler(epoch, learning_rate):
  if epoch < 10:
    return learning_rate
  elif epoch == 10:
    return learning_rate * tensorflow.math.exp(-0.1)
  elif epoch == 20 :
    return learning_rate * tensorflow.math.exp(-0.2)
  elif epoch > 30 :
    return learning_rate * tensorflow.math.exp(-0.2)

learning_scheduler = LearningRateScheduler(scheduler)

Despite the widespread popularity of Adam, recent research papers have noted that it can fail to converge to an optimal solution under specific settings. The paper Improving Generalization Performance by Switching from Adam to SGD demonstrates that adaptive optimization techniques such as Adam generalize poorly compared to SGD.

In [ ]:
# Settings
adam_lr =  0.002#@param {type:"slider", min:0.0001, max:0.01, step:0.0001}
sgd_lr =  0.01#@param {type:"slider", min:0.001, max:0.09, step:0.001}

In [ ]:
# tf.keras.optimizers.Adam(
#     learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
#     name='Adam', **kwargs
# )

# tf.keras.optimizers.SGD(
#     learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD", **kwargs
# )

# Lets tweak learning rate to see rate of conversion
adam = Adam(learning_rate = adam_lr);
sgd = SGD(learning_rate = sgd_lr);
mse_loss = MeanSquaredError();
msle_loss  = MeanSquaredLogarithmicError ();
rmse = RootMeanSquaredError(name='rmse');

logdir = os.path.join("logs", datetime.datetime.today().strftime('%Y-%m-%d-%H:%M'))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

reduce_lr = ReduceLROnPlateau(monitor='rmse', patience=3, verbose=1, factor=0.75, min_lr=0.00001);

early_stop = EarlyStopping(
    monitor="rmse",
    min_delta=0.05,
    patience=10,
    verbose=1,
    mode="min",
    baseline=None,
    restore_best_weights=False,
);

model.compile(loss=mse_loss, optimizer=adam, metrics=['mae',rmse]);

In [ ]:
epochs =  30#@param {type:"slider", min:10, max:300, step:10}

In [ ]:
model.fit(train_gen, epochs=epochs, verbose=1, callbacks=[tensorboard_callback, reduce_lr, early_stop])

In [ ]:
%tensorboard --logdir logs

In [ ]:
model.save( 'my_model' + datetime.datetime.today().strftime('%Y-%m-%d-%H:%M') )

For each Id in the test set, you must predict a probability for the target variable, Pawpularity. The file should contain a header and have the following format:

Id, Pawpularity \
0008dbfb52aa1dc6ee51ee02adf13537, 99.24 \
0014a7b528f1682f0cf3b73a991c17a0, 61.71 \
0019c1388dfcd30ac8b112fb4250c251, 6.23 \
00307b779c82716b240a24f028b0031b, 9.43 \
00320c6dd5b4223c62a9670110d47911, 70.89 \
etc.